# Kernel Learning And Its Application In Nonlinear Support Vector Machines

Sven Nekula (Otto-Friedrich-University Bamberg, sven.nekula@stud.uni-bamberg.de) <br />
Joshua Simon (Otto-Friedrich-University Bamberg, joshua-gunter.simon@stud.uni-bamberg.de)

## Overview
1. Introduction
    * Linear separable data classes
    * Similarity, dot product and vector norm
    * Hyperplane classifiers: Solving an optimization problem
2. Linear SVMs
    * Maximum margin separator
    * Limitations
3. Nonlinear SVMs
    * The kernel trick
    * Solving a nonlinear sepqarable classification problem
4. More kernel applications
5. Conclusion

## Python Moduls

In [2]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Introduction

## Linear separable data classes
First, let's consider a given data set $\mathcal{X}$ of labeled points (inputs) with individual labels $y_i \in \left\{ -1,1 \right\}$, e.g. $(x_1,y_1), ..., (x_m, y_m) \in \mathcal{X} \times \left\{ -1,1 \right\}$. Our goal is to implement a classification method, which is able to classify new and unlabeld data points with the right or "best" label. In machine learning, a well established classification method are the so called __Support Vector Machines__ (SVM). Developed by Vladimir Vapnik and his coworkers in the 1990s, SVMs are still a relevent topic and an even more powerfull tool for __classification__ and __regression__.

In [3]:
def generate_data():
    pass